# Hyperparameter 설명

## 🌳 모델의 전반적인 구조 (Boosting Process)
모델 전체의 학습 방향과 방식을 결정하는 가장 중요한 파라미터입니다.

n_estimators: 생성할 결정 나무의 개수입니다.

비유: 문제를 해결하기 위해 몇 명의 전문가를 투입할지 결정하는 것과 같습니다.

영향: 값이 클수록 모델은 더 정교해지지만, 너무 많으면 과적합(overfitting)될 수 있고 학습 시간이 길어집니다. early_stopping 콜백이 최적의 개수를 찾아주는 역할을 보조합니다.

learning_rate (학습률): 각 나무가 이전 나무의 오류를 얼마나 강하게 보정할지를 나타내는 값입니다. (0~1 사이의 값)

비유: 새로운 전문가의 의견을 얼마나 신뢰하고 반영할지 결정하는 값입니다.

영향: 값이 작을수록 더 많은 n_estimators가 필요하지만, 최적점에 더 안정적으로 도달할 수 있습니다. 값이 크면 학습이 빠르지만, 최적점을 지나쳐 버릴 수 있습니다.

## 🍃 개별 나무의 복잡도 제어 (Tree Complexity)
각각의 나무가 얼마나 복잡한 규칙을 가질 수 있는지를 제어합니다.

num_leaves: 하나의 나무가 가질 수 있는 최대 잎(leaf)의 개수입니다.

비유: 의사결정 순서도에서 나올 수 있는 최종 결론의 최대 가짓수입니다.

영향: 모델의 복잡도를 결정하는 가장 중요한 파라미터 중 하나입니다. 값이 클수록 복잡한 패턴을 학습할 수 있지만, 과적합에 매우 취약해집니다.

max_depth: 나무의 최대 깊이입니다.

비유: 결론에 도달하기까지 최대 몇 단계의 질문을 할 수 있는지 제한하는 것입니다.

영향: 나무가 너무 깊어지는 것을 막아 과적합을 방지합니다. num_leaves와 함께 모델의 복잡도를 제어합니다.

## ⚖️ 과적합 방지를 위한 정규화 (Regularization)
모델이 훈련 데이터에만 너무 치우쳐 학습되는 것을 막는 규제 장치입니다.

lambda_l1 (L1 정규화): 모델의 가중치 합에 페널티를 부여합니다. 특정 피처의 가중치를 0으로 만들어 **피처 선택(feature selection)**의 효과를 낼 수 있습니다.

lambda_l2 (L2 정규화): 모델 가중치의 제곱 합에 페널티를 부여합니다. 가중치 값을 전반적으로 부드럽게 만들어 과적합을 방지합니다.

## 🎲 속도 및 일반화 성능을 위한 샘플링 (Subsampling)
훈련 데이터를 무작위로 샘플링하여 각 나무를 조금씩 다른 데이터로 학습시켜 모델의 일반화 성능을 높입니다.

feature_fraction: 각 나무를 만들 때 무작위로 선택할 피처(열)의 비율입니다. 예를 들어 0.8이면, 매번 80%의 피처만 사용하여 나무를 만듭니다.

bagging_fraction (subsample): 각 나무를 만들 때 무작위로 선택할 데이터(행)의 비율입니다. 예를 들어 0.8이면, 매번 80%의 데이터만 사용합니다.

bagging_freq: 몇 번의 반복(나무 생성)마다 데이터 샘플링을 수행할지 결정합니다. 1이면 매번 새로운 데이터를 샘플링합니다.

# Setting

In [1]:
! pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 28.8 MB/s eta 0:00:00


In [2]:
# === 1. 라이브러리 임포트 ===
import pandas as pd
import numpy as np
import lightgbm as lgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time
import gc  # 🌟 메모리 정리를 위한 가비지 컬렉터 임포트

In [3]:
# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

In [4]:
# --- 데이터 로드 및 전처리 ---
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Sample

In [ ]:
class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/review_helpfulness/PADA/data/amazon/amazon.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/amazon_T5.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

In [ ]:
class EarlyStoppingCallback:
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        # 현재까지의 최고 점수 가져오기
        current_best_value = study.best_value

        # 최고 점수가 갱신되었는지 확인
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0  # 카운터 초기화
        else:
            self._counter += 1 # 점수 갱신 안되면 카운터 증가

        # 정해진 횟수 이상 점수 갱신이 없으면 중단
        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()

def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    params = {
        'objective': 'binary',
        'metric': 'logloss',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'random_state': Config.RANDOM_STATE,
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    }

    # 🌟 GPU 사용을 위해 device='gpu' 파라미터 추가
    model = lgb.LGBMClassifier(device='gpu', **params)

    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              eval_metric='logloss',
              callbacks=[lgb.early_stopping(100, verbose=False)])

    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)

    return score

In [ ]:
config = Config()

In [ ]:
# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71941건, 테스트용: 17986건)


In [ ]:
# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    baseline_model = lgb.LGBMClassifier(device='gpu', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    final_model = lgb.LGBMClassifier(device='gpu', objective='binary', verbosity=-1, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba

    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join(os.path.dirname(config.CSV_FILE_PATH), "T5_amazon_with_s2_predictions.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}' 파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")


📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-11 14:39:21,617] A new study created in memory with name: no-name-f7928362-513a-4cbd-8f07-68750faab16c


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-11 14:39:34,000] Trial 0 finished with value: 0.2956406522336126 and parameters: {'n_estimators': 400, 'learning_rate': 0.03184851035766648, 'num_leaves': 228, 'max_depth': 4, 'lambda_l1': 1.922753540301105, 'lambda_l2': 4.24008297585092, 'feature_fraction': 0.601543595164512, 'bagging_fraction': 0.6456228695497569, 'bagging_freq': 6}. Best is trial 0 with value: 0.2956406522336126.
[I 2025-10-11 14:39:59,101] Trial 1 finished with value: 0.29704213682239977 and parameters: {'n_estimators': 1200, 'learning_rate': 0.015380471142327334, 'num_leaves': 84, 'max_depth': 4, 'lambda_l1': 1.1641702209232268e-07, 'lambda_l2': 9.702731167645007, 'feature_fraction': 0.8030434117039636, 'bagging_fraction': 0.850404055880241, 'bagging_freq': 7}. Best is trial 1 with value: 0.29704213682239977.
[I 2025-10-11 14:40:09,649] Trial 2 finished with value: 0.2907487790108201 and parameters: {'n_estimators': 400, 'learning_rate': 0.034896183909109896, 'num_leaves': 152, 'max_depth': 3, 'lambda_l


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 2000
       learning_rate: 0.0017950086857897345
          num_leaves: 295
           max_depth: 9
           lambda_l1: 0.08387632970052744
           lambda_l2: 8.361277523538194e-05
    feature_fraction: 0.8601866825979766
    bagging_fraction: 0.6193545815988709
        bagging_freq: 4

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.3127   0.7732    0.0997
Optuna Tuned  0.3365   0.7827    0.0700

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/data/amazon/T5_amazon_with_s2_predictions.csv' 파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


# Amazon

## T5

In [ ]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 입력 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/amazon/amazon.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/amazon_T5.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/amazon/T5_amazon_with_s2_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    EARLY_STOPPING_ROUNDS = 10 # 🌟 Optuna 조기 종료 횟수

# === 3. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    """Optuna 스터디의 조기 종료를 위한 콜백 클래스"""
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()

# === 4. Optuna Objective 함수 ===
def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    params = {
        'objective': 'binary', 'metric': 'logloss', 'verbosity': -1,
        'boosting_type': 'gbdt', 'random_state': Config.RANDOM_STATE,
        'device': 'gpu',  # 🌟 Optuna Trial 내부에서도 GPU 사용
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              eval_metric='logloss',
              callbacks=[lgb.early_stopping(100, verbose=False)])

    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)
    return score

# === 5. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # ... (Step 1 ~ 2: 데이터 로드, 분할, 베이스라인 측정) ...
    print("Step 1: 데이터 로드 및 분할 중...")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
        assert len(df) == len(embeddings)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
    )
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    baseline_model = lgb.LGBMClassifier(device='gpu', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 (조기 종료 포함) ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.EARLY_STOPPING_ROUNDS}번 개선 없으면 조기 종료)")
    print("="*50)

    early_stopping_callback = EarlyStoppingCallback(early_stopping_rounds=config.EARLY_STOPPING_ROUNDS)
    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda study, trial: pbar.update(1), early_stopping_callback])
    except optuna.exceptions.OptunaError:
        # 조기 종료 시 발생하는 예외를 정상 처리
        pass
    pbar.close()

    # ... (Step 4 ~ 6: 결과 출력, 저장, 메모리 정리) ...
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    final_model = lgb.LGBMClassifier(device='gpu', objective='binary', verbosity=-1, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    df['s2_pred_proba'] = np.nan
    df['s2_pred_class'] = np.nan

    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[train_indices, 's2_pred_class'] = train_pred_class

    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned
    df.loc[test_indices, 's2_pred_class'] = y_pred_class_tuned

    # 🌟 Config에 설정된 경로로 최종 파일 저장
    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    print("\n" + "="*60)
    print("🧹 Step 7: 사용된 변수들을 메모리에서 정리")
    print("="*60)

    vars_to_delete = [
        'df', 'labels', 'embeddings', 'indices', 'X_train', 'X_test',
        'y_train', 'y_test', 'train_indices', 'test_indices',
        'baseline_model', 'final_model', 'study',
    ]
    for var_name in vars_to_delete:
        if var_name in locals() or var_name in globals():
            if var_name in globals(): del globals()[var_name]

    gc.collect()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")

Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71941건, 테스트용: 17986건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)
[LightGBM] [Info] Number of positive: 7076, number of negative: 64865
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 195840
[LightGBM] [Info] Number of data points in the train set: 71941, number of used features: 768
[LightGBM] [Info] Using GPU Device: NVIDIA A100-SXM4-80GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 768 dense feature groups (52.69 MB) transferred to GPU in 0.041615 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.098358 -> initscore=-2.215599
[LightGBM] [Info] Start training from score -2.215599


[I 2025-10-11 18:33:39,267] A new study created in memory with name: no-name-338b1c2b-b157-4fd8-9b64-b26309478bca


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...
(최대 50번 시도, 10번 개선 없으면 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-11 18:33:49,599] Trial 0 finished with value: 0.29694767669840505 and parameters: {'n_estimators': 1200, 'learning_rate': 0.04915638806701335, 'num_leaves': 70, 'max_depth': 5, 'lambda_l1': 0.00033383742193826386, 'lambda_l2': 3.5192403569765944, 'feature_fraction': 0.785005665093617, 'bagging_fraction': 0.9891244923546361, 'bagging_freq': 1}. Best is trial 0 with value: 0.29694767669840505.
[I 2025-10-11 18:34:00,336] Trial 1 finished with value: 0.2932882557142812 and parameters: {'n_estimators': 400, 'learning_rate': 0.06368537216960567, 'num_leaves': 139, 'max_depth': 6, 'lambda_l1': 0.009511681403698553, 'lambda_l2': 7.928214068625509, 'feature_fraction': 0.7266926359340881, 'bagging_fraction': 0.8249621852756466, 'bagging_freq': 4}. Best is trial 0 with value: 0.29694767669840505.
[I 2025-10-11 18:36:26,748] Trial 2 finished with value: 0.30045609709731175 and parameters: {'n_estimators': 1700, 'learning_rate': 0.003431992492305313, 'num_leaves': 152, 'max_depth': 10, 


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 1500
       learning_rate: 0.005496229720566432
          num_leaves: 292
           max_depth: 12
           lambda_l1: 9.664868582582134
           lambda_l2: 0.0007024265444479108
    feature_fraction: 0.8264808185445718
    bagging_fraction: 0.6023293694775351
        bagging_freq: 1

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.3127   0.7732    0.0997
Optuna Tuned  0.3390   0.7872    0.1031

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/amazon/T5_amazon_with_s2_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 7: 사용된 변수들을 메모리에서 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## BERT

In [ ]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 입력 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/amazon/amazon.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/amazon_BERT.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/amazon/BERT_amazon_with_s2_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    EARLY_STOPPING_ROUNDS = 10 # 🌟 Optuna 조기 종료 횟수

# === 3. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    """Optuna 스터디의 조기 종료를 위한 콜백 클래스"""
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()

# === 4. Optuna Objective 함수 ===
def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    params = {
        'objective': 'binary', 'metric': 'logloss', 'verbosity': -1,
        'boosting_type': 'gbdt', 'random_state': Config.RANDOM_STATE,
        'device': 'gpu',  # 🌟 Optuna Trial 내부에서도 GPU 사용
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              eval_metric='logloss',
              callbacks=[lgb.early_stopping(100, verbose=False)])

    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)
    return score

# === 5. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # ... (Step 1 ~ 2: 데이터 로드, 분할, 베이스라인 측정) ...
    print("Step 1: 데이터 로드 및 분할 중...")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
        assert len(df) == len(embeddings)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
    )
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    baseline_model = lgb.LGBMClassifier(device='gpu', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 (조기 종료 포함) ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.EARLY_STOPPING_ROUNDS}번 개선 없으면 조기 종료)")
    print("="*50)

    early_stopping_callback = EarlyStoppingCallback(early_stopping_rounds=config.EARLY_STOPPING_ROUNDS)
    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda study, trial: pbar.update(1), early_stopping_callback])
    except optuna.exceptions.OptunaError:
        # 조기 종료 시 발생하는 예외를 정상 처리
        pass
    pbar.close()

    # ... (Step 4 ~ 6: 결과 출력, 저장, 메모리 정리) ...
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    final_model = lgb.LGBMClassifier(device='gpu', objective='binary', verbosity=-1, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    df['s2_pred_proba'] = np.nan
    df['s2_pred_class'] = np.nan

    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[train_indices, 's2_pred_class'] = train_pred_class

    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned
    df.loc[test_indices, 's2_pred_class'] = y_pred_class_tuned

    # 🌟 Config에 설정된 경로로 최종 파일 저장
    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    print("\n" + "="*60)
    print("🧹 Step 7: 사용된 변수들을 메모리에서 정리")
    print("="*60)

    vars_to_delete = [
        'df', 'labels', 'embeddings', 'indices', 'X_train', 'X_test',
        'y_train', 'y_test', 'train_indices', 'test_indices',
        'baseline_model', 'final_model', 'study',
    ]
    for var_name in vars_to_delete:
        if var_name in locals() or var_name in globals():
            if var_name in globals(): del globals()[var_name]

    gc.collect()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")

Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71941건, 테스트용: 17986건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-11 18:57:46,820] A new study created in memory with name: no-name-478c9d9f-9ad4-41b3-a5fb-08060085e135


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...
(최대 50번 시도, 10번 개선 없으면 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-11 18:58:02,987] Trial 0 finished with value: 0.242930227317717 and parameters: {'n_estimators': 600, 'learning_rate': 0.2011606089965192, 'num_leaves': 188, 'max_depth': 9, 'lambda_l1': 0.000655527579171505, 'lambda_l2': 3.3615343618624967e-06, 'feature_fraction': 0.7982463389762611, 'bagging_fraction': 0.7743520732714457, 'bagging_freq': 2}. Best is trial 0 with value: 0.242930227317717.
[I 2025-10-11 18:59:10,883] Trial 1 finished with value: 0.2904482025055546 and parameters: {'n_estimators': 600, 'learning_rate': 0.005222081618070164, 'num_leaves': 142, 'max_depth': 10, 'lambda_l1': 8.452617646318008e-06, 'lambda_l2': 2.2463890305510763e-06, 'feature_fraction': 0.8315848628818452, 'bagging_fraction': 0.9762653282723361, 'bagging_freq': 4}. Best is trial 1 with value: 0.2904482025055546.
[I 2025-10-11 18:59:30,572] Trial 2 finished with value: 0.28736341910265883 and parameters: {'n_estimators': 500, 'learning_rate': 0.0011256116717559155, 'num_leaves': 188, 'max_depth':


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 1000
       learning_rate: 0.011748629338705472
          num_leaves: 29
           max_depth: 5
           lambda_l1: 6.047446186257771
           lambda_l2: 2.6718555871817493e-05
    feature_fraction: 0.7865464309584396
    bagging_fraction: 0.8344669085489922
        bagging_freq: 7

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.3152   0.7701    0.1401
Optuna Tuned  0.3274   0.7783    0.1373

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/amazon/BERT_amazon_with_s2_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 7: 사용된 변수들을 메모리에서 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## SentenceBERT

In [ ]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 입력 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/amazon/amazon.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/amazon_SentenceBERT.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/amazon/SentenceBERT_amazon_with_s2_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    EARLY_STOPPING_ROUNDS = 10 # 🌟 Optuna 조기 종료 횟수

# === 3. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    """Optuna 스터디의 조기 종료를 위한 콜백 클래스"""
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()

# === 4. Optuna Objective 함수 ===
def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    params = {
        'objective': 'binary', 'metric': 'logloss', 'verbosity': -1,
        'boosting_type': 'gbdt', 'random_state': Config.RANDOM_STATE,
        'device': 'gpu',  # 🌟 Optuna Trial 내부에서도 GPU 사용
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              eval_metric='logloss',
              callbacks=[lgb.early_stopping(100, verbose=False)])

    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)
    return score

# === 5. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # ... (Step 1 ~ 2: 데이터 로드, 분할, 베이스라인 측정) ...
    print("Step 1: 데이터 로드 및 분할 중...")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
        assert len(df) == len(embeddings)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
    )
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    baseline_model = lgb.LGBMClassifier(device='gpu', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 (조기 종료 포함) ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.EARLY_STOPPING_ROUNDS}번 개선 없으면 조기 종료)")
    print("="*50)

    early_stopping_callback = EarlyStoppingCallback(early_stopping_rounds=config.EARLY_STOPPING_ROUNDS)
    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda study, trial: pbar.update(1), early_stopping_callback])
    except optuna.exceptions.OptunaError:
        # 조기 종료 시 발생하는 예외를 정상 처리
        pass
    pbar.close()

    # ... (Step 4 ~ 6: 결과 출력, 저장, 메모리 정리) ...
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    final_model = lgb.LGBMClassifier(device='gpu', objective='binary', verbosity=-1, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    df['s2_pred_proba'] = np.nan
    df['s2_pred_class'] = np.nan

    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[train_indices, 's2_pred_class'] = train_pred_class

    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned
    df.loc[test_indices, 's2_pred_class'] = y_pred_class_tuned

    # 🌟 Config에 설정된 경로로 최종 파일 저장
    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    print("\n" + "="*60)
    print("🧹 Step 7: 사용된 변수들을 메모리에서 정리")
    print("="*60)

    vars_to_delete = [
        'df', 'labels', 'embeddings', 'indices', 'X_train', 'X_test',
        'y_train', 'y_test', 'train_indices', 'test_indices',
        'baseline_model', 'final_model', 'study',
    ]
    for var_name in vars_to_delete:
        if var_name in locals() or var_name in globals():
            if var_name in globals(): del globals()[var_name]

    gc.collect()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")

Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71941건, 테스트용: 17986건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-11 19:11:41,975] A new study created in memory with name: no-name-ff435be8-99d5-469c-8a3a-473ff8811f32


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...
(최대 50번 시도, 10번 개선 없으면 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-11 19:11:54,132] Trial 0 finished with value: 0.2512513326101477 and parameters: {'n_estimators': 1600, 'learning_rate': 0.10164272201141614, 'num_leaves': 280, 'max_depth': 11, 'lambda_l1': 8.580120834098538e-07, 'lambda_l2': 0.7344840124454822, 'feature_fraction': 0.6530012516311727, 'bagging_fraction': 0.6301144516663705, 'bagging_freq': 3}. Best is trial 0 with value: 0.2512513326101477.
[I 2025-10-11 19:12:51,213] Trial 1 finished with value: 0.28817652242174085 and parameters: {'n_estimators': 1700, 'learning_rate': 0.004970170835827217, 'num_leaves': 240, 'max_depth': 8, 'lambda_l1': 0.0019450267592229246, 'lambda_l2': 6.0336511125194645e-05, 'feature_fraction': 0.9265819608159555, 'bagging_fraction': 0.8769593277719894, 'bagging_freq': 2}. Best is trial 1 with value: 0.28817652242174085.
[I 2025-10-11 19:13:02,994] Trial 2 finished with value: 0.2321169196744183 and parameters: {'n_estimators': 400, 'learning_rate': 0.16304016337811084, 'num_leaves': 182, 'max_depth'


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 1700
       learning_rate: 0.004970170835827217
          num_leaves: 240
           max_depth: 8
           lambda_l1: 0.0019450267592229246
           lambda_l2: 6.0336511125194645e-05
    feature_fraction: 0.9265819608159555
    bagging_fraction: 0.8769593277719894
        bagging_freq: 2

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.3003   0.7642    0.0931
Optuna Tuned  0.3186   0.7751    0.0875

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/amazon/SentenceBERT_amazon_with_s2_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 7: 사용된 변수들을 메모리에서 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## RoBERTa

In [ ]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 입력 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/amazon/amazon.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/amazon_RoBERTa.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/amazon/RoBERTa_amazon_with_s2_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    EARLY_STOPPING_ROUNDS = 10 # 🌟 Optuna 조기 종료 횟수

# === 3. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    """Optuna 스터디의 조기 종료를 위한 콜백 클래스"""
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()

# === 4. Optuna Objective 함수 ===
def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    params = {
        'objective': 'binary', 'metric': 'logloss', 'verbosity': -1,
        'boosting_type': 'gbdt', 'random_state': Config.RANDOM_STATE,
        'device': 'gpu',  # 🌟 Optuna Trial 내부에서도 GPU 사용
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              eval_metric='logloss',
              callbacks=[lgb.early_stopping(100, verbose=False)])

    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)
    return score

# === 5. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # ... (Step 1 ~ 2: 데이터 로드, 분할, 베이스라인 측정) ...
    print("Step 1: 데이터 로드 및 분할 중...")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
        assert len(df) == len(embeddings)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
    )
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    baseline_model = lgb.LGBMClassifier(device='gpu', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 (조기 종료 포함) ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.EARLY_STOPPING_ROUNDS}번 개선 없으면 조기 종료)")
    print("="*50)

    early_stopping_callback = EarlyStoppingCallback(early_stopping_rounds=config.EARLY_STOPPING_ROUNDS)
    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda study, trial: pbar.update(1), early_stopping_callback])
    except optuna.exceptions.OptunaError:
        # 조기 종료 시 발생하는 예외를 정상 처리
        pass
    pbar.close()

    # ... (Step 4 ~ 6: 결과 출력, 저장, 메모리 정리) ...
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    final_model = lgb.LGBMClassifier(device='gpu', objective='binary', verbosity=-1, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    df['s2_pred_proba'] = np.nan
    df['s2_pred_class'] = np.nan

    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[train_indices, 's2_pred_class'] = train_pred_class

    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned
    df.loc[test_indices, 's2_pred_class'] = y_pred_class_tuned

    # 🌟 Config에 설정된 경로로 최종 파일 저장
    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    print("\n" + "="*60)
    print("🧹 Step 7: 사용된 변수들을 메모리에서 정리")
    print("="*60)

    vars_to_delete = [
        'df', 'labels', 'embeddings', 'indices', 'X_train', 'X_test',
        'y_train', 'y_test', 'train_indices', 'test_indices',
        'baseline_model', 'final_model', 'study',
    ]
    for var_name in vars_to_delete:
        if var_name in locals() or var_name in globals():
            if var_name in globals(): del globals()[var_name]

    gc.collect()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")

Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71941건, 테스트용: 17986건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-11 19:16:15,357] A new study created in memory with name: no-name-6e2b4125-4bb0-4706-b2d8-f37c80d0fe44


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...
(최대 50번 시도, 10번 개선 없으면 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-11 19:20:28,274] Trial 0 finished with value: 0.2845765103552348 and parameters: {'n_estimators': 1300, 'learning_rate': 0.0022185345762837753, 'num_leaves': 254, 'max_depth': 12, 'lambda_l1': 0.00011466209579466375, 'lambda_l2': 1.4861703991610546e-06, 'feature_fraction': 0.9392497545512575, 'bagging_fraction': 0.8876835171895672, 'bagging_freq': 2}. Best is trial 0 with value: 0.2845765103552348.
[I 2025-10-11 19:20:57,127] Trial 1 finished with value: 0.28028550444176525 and parameters: {'n_estimators': 400, 'learning_rate': 0.04312195349220553, 'num_leaves': 237, 'max_depth': 12, 'lambda_l1': 2.17003435760872e-07, 'lambda_l2': 1.1205508654860938, 'feature_fraction': 0.7333691271136558, 'bagging_fraction': 0.9626757765544744, 'bagging_freq': 1}. Best is trial 0 with value: 0.2845765103552348.
[I 2025-10-11 19:21:14,535] Trial 2 finished with value: 0.2881075799334142 and parameters: {'n_estimators': 1100, 'learning_rate': 0.007148434987831936, 'num_leaves': 258, 'max_dept


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 1600
       learning_rate: 0.004736672222231734
          num_leaves: 165
           max_depth: 5
           lambda_l1: 1.0216786129101437e-07
           lambda_l2: 8.246206027329432
    feature_fraction: 0.6672178520176764
    bagging_fraction: 0.7247545118086824
        bagging_freq: 5

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.3155   0.7633    0.1465
Optuna Tuned  0.3301   0.7756    0.1354

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/amazon/RoBERTa_amazon_with_s2_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 7: 사용된 변수들을 메모리에서 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## DistilBERT

In [ ]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 입력 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/amazon/amazon.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/amazon_DistilBERT.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/amazon/DistilBERT_amazon_with_s2_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    EARLY_STOPPING_ROUNDS = 10 # 🌟 Optuna 조기 종료 횟수

# === 3. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    """Optuna 스터디의 조기 종료를 위한 콜백 클래스"""
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()

# === 4. Optuna Objective 함수 ===
def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    params = {
        'objective': 'binary', 'metric': 'logloss', 'verbosity': -1,
        'boosting_type': 'gbdt', 'random_state': Config.RANDOM_STATE,
        'device': 'gpu',  # 🌟 Optuna Trial 내부에서도 GPU 사용
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              eval_metric='logloss',
              callbacks=[lgb.early_stopping(100, verbose=False)])

    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)
    return score

# === 5. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # ... (Step 1 ~ 2: 데이터 로드, 분할, 베이스라인 측정) ...
    print("Step 1: 데이터 로드 및 분할 중...")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
        assert len(df) == len(embeddings)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
    )
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    baseline_model = lgb.LGBMClassifier(device='gpu', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 (조기 종료 포함) ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.EARLY_STOPPING_ROUNDS}번 개선 없으면 조기 종료)")
    print("="*50)

    early_stopping_callback = EarlyStoppingCallback(early_stopping_rounds=config.EARLY_STOPPING_ROUNDS)
    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda study, trial: pbar.update(1), early_stopping_callback])
    except optuna.exceptions.OptunaError:
        # 조기 종료 시 발생하는 예외를 정상 처리
        pass
    pbar.close()

    # ... (Step 4 ~ 6: 결과 출력, 저장, 메모리 정리) ...
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    final_model = lgb.LGBMClassifier(device='gpu', objective='binary', verbosity=-1, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    df['s2_pred_proba'] = np.nan
    df['s2_pred_class'] = np.nan

    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[train_indices, 's2_pred_class'] = train_pred_class

    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned
    df.loc[test_indices, 's2_pred_class'] = y_pred_class_tuned

    # 🌟 Config에 설정된 경로로 최종 파일 저장
    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    print("\n" + "="*60)
    print("🧹 Step 7: 사용된 변수들을 메모리에서 정리")
    print("="*60)

    vars_to_delete = [
        'df', 'labels', 'embeddings', 'indices', 'X_train', 'X_test',
        'y_train', 'y_test', 'train_indices', 'test_indices',
        'baseline_model', 'final_model', 'study',
    ]
    for var_name in vars_to_delete:
        if var_name in locals() or var_name in globals():
            if var_name in globals(): del globals()[var_name]

    gc.collect()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")

Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71941건, 테스트용: 17986건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-11 19:48:03,839] A new study created in memory with name: no-name-87f04557-6a25-4268-bb7e-b274394d7c19


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...
(최대 50번 시도, 10번 개선 없으면 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-11 19:49:23,622] Trial 0 finished with value: 0.29355160766737765 and parameters: {'n_estimators': 900, 'learning_rate': 0.009295730322749328, 'num_leaves': 277, 'max_depth': 12, 'lambda_l1': 3.012658205207445e-05, 'lambda_l2': 1.0215215876064099, 'feature_fraction': 0.8316498356072956, 'bagging_fraction': 0.9139956440420921, 'bagging_freq': 1}. Best is trial 0 with value: 0.29355160766737765.
[I 2025-10-11 19:50:11,099] Trial 1 finished with value: 0.28981896281638114 and parameters: {'n_estimators': 1700, 'learning_rate': 0.007611179949916501, 'num_leaves': 76, 'max_depth': 10, 'lambda_l1': 0.00036541558170006454, 'lambda_l2': 0.0015393019511521647, 'feature_fraction': 0.875899037692794, 'bagging_fraction': 0.9530256503958171, 'bagging_freq': 2}. Best is trial 0 with value: 0.29355160766737765.
[I 2025-10-11 19:50:16,799] Trial 2 finished with value: 0.2775046981069139 and parameters: {'n_estimators': 200, 'learning_rate': 0.008285495992609285, 'num_leaves': 104, 'max_dept


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 900
       learning_rate: 0.009295730322749328
          num_leaves: 277
           max_depth: 12
           lambda_l1: 3.012658205207445e-05
           lambda_l2: 1.0215215876064099
    feature_fraction: 0.8316498356072956
    bagging_fraction: 0.9139956440420921
        bagging_freq: 1

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.3090   0.7732    0.1364
Optuna Tuned  0.3259   0.7785    0.1296

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/amazon/DistilBERT_amazon_with_s2_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 7: 사용된 변수들을 메모리에서 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


# Coursera

## T5

In [ ]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 입력 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/coursera/coursera.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/coursera_T5.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/T5_coursera_with_s2_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    EARLY_STOPPING_ROUNDS = 10 # 🌟 Optuna 조기 종료 횟수

# === 3. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    """Optuna 스터디의 조기 종료를 위한 콜백 클래스"""
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()

# === 4. Optuna Objective 함수 ===
def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    params = {
        'objective': 'binary', 'metric': 'logloss', 'verbosity': -1,
        'boosting_type': 'gbdt', 'random_state': Config.RANDOM_STATE,
        'device': 'gpu',  # 🌟 Optuna Trial 내부에서도 GPU 사용
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              eval_metric='logloss',
              callbacks=[lgb.early_stopping(100, verbose=False)])

    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)
    return score

# === 5. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # ... (Step 1 ~ 2: 데이터 로드, 분할, 베이스라인 측정) ...
    print("Step 1: 데이터 로드 및 분할 중...")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
        assert len(df) == len(embeddings)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
    )
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    baseline_model = lgb.LGBMClassifier(device='gpu', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 (조기 종료 포함) ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.EARLY_STOPPING_ROUNDS}번 개선 없으면 조기 종료)")
    print("="*50)

    early_stopping_callback = EarlyStoppingCallback(early_stopping_rounds=config.EARLY_STOPPING_ROUNDS)
    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda study, trial: pbar.update(1), early_stopping_callback])
    except optuna.exceptions.OptunaError:
        # 조기 종료 시 발생하는 예외를 정상 처리
        pass
    pbar.close()

    # ... (Step 4 ~ 6: 결과 출력, 저장, 메모리 정리) ...
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    final_model = lgb.LGBMClassifier(device='gpu', objective='binary', verbosity=-1, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    df['s2_pred_proba'] = np.nan
    df['s2_pred_class'] = np.nan

    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[train_indices, 's2_pred_class'] = train_pred_class

    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned
    df.loc[test_indices, 's2_pred_class'] = y_pred_class_tuned

    # 🌟 Config에 설정된 경로로 최종 파일 저장
    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    print("\n" + "="*60)
    print("🧹 Step 7: 사용된 변수들을 메모리에서 정리")
    print("="*60)

    vars_to_delete = [
        'df', 'labels', 'embeddings', 'indices', 'X_train', 'X_test',
        'y_train', 'y_test', 'train_indices', 'test_indices',
        'baseline_model', 'final_model', 'study',
    ]
    for var_name in vars_to_delete:
        if var_name in locals() or var_name in globals():
            if var_name in globals(): del globals()[var_name]

    gc.collect()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")

Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 97108건, 테스트용: 24278건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-11 19:58:14,839] A new study created in memory with name: no-name-def89859-c8b9-4b6a-a916-79ceec6b97a7


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...
(최대 50번 시도, 10번 개선 없으면 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-11 19:58:52,379] Trial 0 finished with value: 0.21666437305240877 and parameters: {'n_estimators': 1200, 'learning_rate': 0.01637537335615552, 'num_leaves': 117, 'max_depth': 12, 'lambda_l1': 6.856253928276642e-05, 'lambda_l2': 4.898478761629054e-07, 'feature_fraction': 0.9192453560829086, 'bagging_fraction': 0.8581375935582478, 'bagging_freq': 7}. Best is trial 0 with value: 0.21666437305240877.
[I 2025-10-11 19:59:05,170] Trial 1 finished with value: 0.17487312258815896 and parameters: {'n_estimators': 500, 'learning_rate': 0.12511384415385518, 'num_leaves': 233, 'max_depth': 7, 'lambda_l1': 5.755964763444149e-06, 'lambda_l2': 4.174804057710874e-08, 'feature_fraction': 0.886551297588371, 'bagging_fraction': 0.742379760892753, 'bagging_freq': 2}. Best is trial 0 with value: 0.21666437305240877.
[I 2025-10-11 20:02:14,445] Trial 2 finished with value: 0.2212457955590728 and parameters: {'n_estimators': 1400, 'learning_rate': 0.0014421002981804564, 'num_leaves': 182, 'max_dep


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 1600
       learning_rate: 0.0036738077327695073
          num_leaves: 79
           max_depth: 11
           lambda_l1: 1.0074209283711079e-08
           lambda_l2: 0.0011833529803131166
    feature_fraction: 0.9543663161446166
    bagging_fraction: 0.9363123245517312
        bagging_freq: 4

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.2038   0.7835    0.0875
Optuna Tuned  0.2279   0.7967    0.0646

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/T5_coursera_with_s2_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 7: 사용된 변수들을 메모리에서 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## BERT

In [ ]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 입력 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/coursera/coursera.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/coursera_BERT.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/BERT_coursera_with_s2_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    EARLY_STOPPING_ROUNDS = 10 # 🌟 Optuna 조기 종료 횟수

# === 3. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    """Optuna 스터디의 조기 종료를 위한 콜백 클래스"""
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()

# === 4. Optuna Objective 함수 ===
def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    params = {
        'objective': 'binary', 'metric': 'logloss', 'verbosity': -1,
        'boosting_type': 'gbdt', 'random_state': Config.RANDOM_STATE,
        'device': 'gpu',  # 🌟 Optuna Trial 내부에서도 GPU 사용
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              eval_metric='logloss',
              callbacks=[lgb.early_stopping(100, verbose=False)])

    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)
    return score

# === 5. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # ... (Step 1 ~ 2: 데이터 로드, 분할, 베이스라인 측정) ...
    print("Step 1: 데이터 로드 및 분할 중...")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
        assert len(df) == len(embeddings)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
    )
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    baseline_model = lgb.LGBMClassifier(device='gpu', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 (조기 종료 포함) ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.EARLY_STOPPING_ROUNDS}번 개선 없으면 조기 종료)")
    print("="*50)

    early_stopping_callback = EarlyStoppingCallback(early_stopping_rounds=config.EARLY_STOPPING_ROUNDS)
    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda study, trial: pbar.update(1), early_stopping_callback])
    except optuna.exceptions.OptunaError:
        # 조기 종료 시 발생하는 예외를 정상 처리
        pass
    pbar.close()

    # ... (Step 4 ~ 6: 결과 출력, 저장, 메모리 정리) ...
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    final_model = lgb.LGBMClassifier(device='gpu', objective='binary', verbosity=-1, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    df['s2_pred_proba'] = np.nan
    df['s2_pred_class'] = np.nan

    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[train_indices, 's2_pred_class'] = train_pred_class

    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned
    df.loc[test_indices, 's2_pred_class'] = y_pred_class_tuned

    # 🌟 Config에 설정된 경로로 최종 파일 저장
    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    print("\n" + "="*60)
    print("🧹 Step 7: 사용된 변수들을 메모리에서 정리")
    print("="*60)

    vars_to_delete = [
        'df', 'labels', 'embeddings', 'indices', 'X_train', 'X_test',
        'y_train', 'y_test', 'train_indices', 'test_indices',
        'baseline_model', 'final_model', 'study',
    ]
    for var_name in vars_to_delete:
        if var_name in locals() or var_name in globals():
            if var_name in globals(): del globals()[var_name]

    gc.collect()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")

Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 97108건, 테스트용: 24278건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-11 20:25:02,209] A new study created in memory with name: no-name-ef9c70cd-a5b8-4cc9-bd31-ae9454733e92


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...
(최대 50번 시도, 10번 개선 없으면 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-11 20:25:15,995] Trial 0 finished with value: 0.1961827588130428 and parameters: {'n_estimators': 1100, 'learning_rate': 0.0048882465744390455, 'num_leaves': 120, 'max_depth': 3, 'lambda_l1': 0.2566080454504444, 'lambda_l2': 7.820734317823883e-07, 'feature_fraction': 0.8208990572375937, 'bagging_fraction': 0.8626004829490532, 'bagging_freq': 6}. Best is trial 0 with value: 0.1961827588130428.
[I 2025-10-11 20:25:34,835] Trial 1 finished with value: 0.19621297572660756 and parameters: {'n_estimators': 1200, 'learning_rate': 0.0034201821519429768, 'num_leaves': 133, 'max_depth': 4, 'lambda_l1': 4.966310551658748, 'lambda_l2': 8.374952468781421, 'feature_fraction': 0.7198332737984545, 'bagging_fraction': 0.6135450296561967, 'bagging_freq': 2}. Best is trial 1 with value: 0.19621297572660756.
[I 2025-10-11 20:25:44,492] Trial 2 finished with value: 0.14374891780567803 and parameters: {'n_estimators': 1400, 'learning_rate': 0.23457442196838957, 'num_leaves': 34, 'max_depth': 9, '


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 500
       learning_rate: 0.011918054443761877
          num_leaves: 113
           max_depth: 10
           lambda_l1: 0.0007441469827980438
           lambda_l2: 4.252650394325436
    feature_fraction: 0.6225137409030421
    bagging_fraction: 0.7551596115189563
        bagging_freq: 3

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1819   0.7689    0.0893
Optuna Tuned  0.2140   0.7886    0.0647

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/BERT_coursera_with_s2_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 7: 사용된 변수들을 메모리에서 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## SentenceBERT

In [ ]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 입력 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/coursera/coursera.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/coursera_SentenceBERT.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/SentenceBERT_coursera_with_s2_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    EARLY_STOPPING_ROUNDS = 10 # 🌟 Optuna 조기 종료 횟수

# === 3. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    """Optuna 스터디의 조기 종료를 위한 콜백 클래스"""
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()

# === 4. Optuna Objective 함수 ===
def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    params = {
        'objective': 'binary', 'metric': 'logloss', 'verbosity': -1,
        'boosting_type': 'gbdt', 'random_state': Config.RANDOM_STATE,
        'device': 'gpu',  # 🌟 Optuna Trial 내부에서도 GPU 사용
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              eval_metric='logloss',
              callbacks=[lgb.early_stopping(100, verbose=False)])

    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)
    return score

# === 5. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # ... (Step 1 ~ 2: 데이터 로드, 분할, 베이스라인 측정) ...
    print("Step 1: 데이터 로드 및 분할 중...")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
        assert len(df) == len(embeddings)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
    )
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    baseline_model = lgb.LGBMClassifier(device='gpu', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 (조기 종료 포함) ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.EARLY_STOPPING_ROUNDS}번 개선 없으면 조기 종료)")
    print("="*50)

    early_stopping_callback = EarlyStoppingCallback(early_stopping_rounds=config.EARLY_STOPPING_ROUNDS)
    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda study, trial: pbar.update(1), early_stopping_callback])
    except optuna.exceptions.OptunaError:
        # 조기 종료 시 발생하는 예외를 정상 처리
        pass
    pbar.close()

    # ... (Step 4 ~ 6: 결과 출력, 저장, 메모리 정리) ...
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    final_model = lgb.LGBMClassifier(device='gpu', objective='binary', verbosity=-1, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    df['s2_pred_proba'] = np.nan
    df['s2_pred_class'] = np.nan

    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[train_indices, 's2_pred_class'] = train_pred_class

    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned
    df.loc[test_indices, 's2_pred_class'] = y_pred_class_tuned

    # 🌟 Config에 설정된 경로로 최종 파일 저장
    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    print("\n" + "="*60)
    print("🧹 Step 7: 사용된 변수들을 메모리에서 정리")
    print("="*60)

    vars_to_delete = [
        'df', 'labels', 'embeddings', 'indices', 'X_train', 'X_test',
        'y_train', 'y_test', 'train_indices', 'test_indices',
        'baseline_model', 'final_model', 'study',
    ]
    for var_name in vars_to_delete:
        if var_name in locals() or var_name in globals():
            if var_name in globals(): del globals()[var_name]

    gc.collect()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")

Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 97108건, 테스트용: 24278건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-11 20:37:14,777] A new study created in memory with name: no-name-f274ba56-50f1-47c0-8e1a-cd366b8dc6b3


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...
(최대 50번 시도, 10번 개선 없으면 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-11 20:37:44,011] Trial 0 finished with value: 0.20022188125795112 and parameters: {'n_estimators': 1300, 'learning_rate': 0.002276460191144249, 'num_leaves': 249, 'max_depth': 6, 'lambda_l1': 7.941848998960709e-08, 'lambda_l2': 1.0250116917434139e-05, 'feature_fraction': 0.7708413003197726, 'bagging_fraction': 0.826954036942413, 'bagging_freq': 7}. Best is trial 0 with value: 0.20022188125795112.
[I 2025-10-11 20:38:11,653] Trial 1 finished with value: 0.1976726415702342 and parameters: {'n_estimators': 1400, 'learning_rate': 0.016343904041966062, 'num_leaves': 196, 'max_depth': 12, 'lambda_l1': 0.00023624182131236414, 'lambda_l2': 0.0014459001102957738, 'feature_fraction': 0.8471742731822693, 'bagging_fraction': 0.8984822288045906, 'bagging_freq': 2}. Best is trial 0 with value: 0.20022188125795112.
[I 2025-10-11 20:38:28,994] Trial 2 finished with value: 0.18586545548066655 and parameters: {'n_estimators': 1800, 'learning_rate': 0.001404353208653213, 'num_leaves': 197, 'ma


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 2000
       learning_rate: 0.00411758295662484
          num_leaves: 137
           max_depth: 5
           lambda_l1: 0.0002660818959315374
           lambda_l2: 1.2115732946146658
    feature_fraction: 0.9300570317392396
    bagging_fraction: 0.6379945563339555
        bagging_freq: 4

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1885   0.7751    0.0811
Optuna Tuned  0.2139   0.7872    0.0515

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/SentenceBERT_coursera_with_s2_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 7: 사용된 변수들을 메모리에서 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## RoBERTa

In [ ]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 입력 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/coursera/coursera.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/coursera_RoBERTa.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/RoBERTa_coursera_with_s2_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    EARLY_STOPPING_ROUNDS = 10 # 🌟 Optuna 조기 종료 횟수

# === 3. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    """Optuna 스터디의 조기 종료를 위한 콜백 클래스"""
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()

# === 4. Optuna Objective 함수 ===
def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    params = {
        'objective': 'binary', 'metric': 'logloss', 'verbosity': -1,
        'boosting_type': 'gbdt', 'random_state': Config.RANDOM_STATE,
        'device': 'gpu',  # 🌟 Optuna Trial 내부에서도 GPU 사용
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              eval_metric='logloss',
              callbacks=[lgb.early_stopping(100, verbose=False)])

    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)
    return score

# === 5. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # ... (Step 1 ~ 2: 데이터 로드, 분할, 베이스라인 측정) ...
    print("Step 1: 데이터 로드 및 분할 중...")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
        assert len(df) == len(embeddings)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
    )
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    baseline_model = lgb.LGBMClassifier(device='gpu', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 (조기 종료 포함) ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.EARLY_STOPPING_ROUNDS}번 개선 없으면 조기 종료)")
    print("="*50)

    early_stopping_callback = EarlyStoppingCallback(early_stopping_rounds=config.EARLY_STOPPING_ROUNDS)
    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda study, trial: pbar.update(1), early_stopping_callback])
    except optuna.exceptions.OptunaError:
        # 조기 종료 시 발생하는 예외를 정상 처리
        pass
    pbar.close()

    # ... (Step 4 ~ 6: 결과 출력, 저장, 메모리 정리) ...
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    final_model = lgb.LGBMClassifier(device='gpu', objective='binary', verbosity=-1, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    df['s2_pred_proba'] = np.nan
    df['s2_pred_class'] = np.nan

    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[train_indices, 's2_pred_class'] = train_pred_class

    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned
    df.loc[test_indices, 's2_pred_class'] = y_pred_class_tuned

    # 🌟 Config에 설정된 경로로 최종 파일 저장
    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    print("\n" + "="*60)
    print("🧹 Step 7: 사용된 변수들을 메모리에서 정리")
    print("="*60)

    vars_to_delete = [
        'df', 'labels', 'embeddings', 'indices', 'X_train', 'X_test',
        'y_train', 'y_test', 'train_indices', 'test_indices',
        'baseline_model', 'final_model', 'study',
    ]
    for var_name in vars_to_delete:
        if var_name in locals() or var_name in globals():
            if var_name in globals(): del globals()[var_name]

    gc.collect()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")

Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 97108건, 테스트용: 24278건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-11 20:45:44,523] A new study created in memory with name: no-name-d6c8b90d-1760-48ec-ad57-a7ad6f60352a


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...
(최대 50번 시도, 10번 개선 없으면 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-11 20:45:52,090] Trial 0 finished with value: 0.17019138373960713 and parameters: {'n_estimators': 700, 'learning_rate': 0.17584144746091818, 'num_leaves': 82, 'max_depth': 4, 'lambda_l1': 4.339008005892517, 'lambda_l2': 0.02079242601796223, 'feature_fraction': 0.7943961375137776, 'bagging_fraction': 0.63106581538639, 'bagging_freq': 7}. Best is trial 0 with value: 0.17019138373960713.
[I 2025-10-11 20:46:36,817] Trial 1 finished with value: 0.18541249803635354 and parameters: {'n_estimators': 400, 'learning_rate': 0.004841601222991729, 'num_leaves': 138, 'max_depth': 8, 'lambda_l1': 5.331940591085429e-08, 'lambda_l2': 9.384495553569026e-05, 'feature_fraction': 0.9364816937541144, 'bagging_fraction': 0.9657035929252764, 'bagging_freq': 5}. Best is trial 1 with value: 0.18541249803635354.
[I 2025-10-11 20:47:13,731] Trial 2 finished with value: 0.17026062436177236 and parameters: {'n_estimators': 200, 'learning_rate': 0.0022623264563756784, 'num_leaves': 210, 'max_depth': 10,


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 800
       learning_rate: 0.04491713933013319
          num_leaves: 64
           max_depth: 3
           lambda_l1: 5.046608656123682e-08
           lambda_l2: 5.90361203462853
    feature_fraction: 0.7677137700830172
    bagging_fraction: 0.656918951624284
        bagging_freq: 2

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1784   0.7533    0.0818
Optuna Tuned  0.2066   0.7668    0.0821

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/RoBERTa_coursera_with_s2_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 7: 사용된 변수들을 메모리에서 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## DistilBERT

In [ ]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 입력 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/coursera/coursera.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/coursera_DistilBERT.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/DistilBERT_coursera_with_s2_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    EARLY_STOPPING_ROUNDS = 10 # 🌟 Optuna 조기 종료 횟수

# === 3. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    """Optuna 스터디의 조기 종료를 위한 콜백 클래스"""
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()

# === 4. Optuna Objective 함수 ===
def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    params = {
        'objective': 'binary', 'metric': 'logloss', 'verbosity': -1,
        'boosting_type': 'gbdt', 'random_state': Config.RANDOM_STATE,
        'device': 'gpu',  # 🌟 Optuna Trial 내부에서도 GPU 사용
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              eval_metric='logloss',
              callbacks=[lgb.early_stopping(100, verbose=False)])

    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)
    return score

# === 5. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # ... (Step 1 ~ 2: 데이터 로드, 분할, 베이스라인 측정) ...
    print("Step 1: 데이터 로드 및 분할 중...")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
        assert len(df) == len(embeddings)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
    )
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    baseline_model = lgb.LGBMClassifier(device='gpu', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 (조기 종료 포함) ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.EARLY_STOPPING_ROUNDS}번 개선 없으면 조기 종료)")
    print("="*50)

    early_stopping_callback = EarlyStoppingCallback(early_stopping_rounds=config.EARLY_STOPPING_ROUNDS)
    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda study, trial: pbar.update(1), early_stopping_callback])
    except optuna.exceptions.OptunaError:
        # 조기 종료 시 발생하는 예외를 정상 처리
        pass
    pbar.close()

    # ... (Step 4 ~ 6: 결과 출력, 저장, 메모리 정리) ...
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    final_model = lgb.LGBMClassifier(device='gpu', objective='binary', verbosity=-1, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    df['s2_pred_proba'] = np.nan
    df['s2_pred_class'] = np.nan

    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[train_indices, 's2_pred_class'] = train_pred_class

    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned
    df.loc[test_indices, 's2_pred_class'] = y_pred_class_tuned

    # 🌟 Config에 설정된 경로로 최종 파일 저장
    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    print("\n" + "="*60)
    print("🧹 Step 7: 사용된 변수들을 메모리에서 정리")
    print("="*60)

    vars_to_delete = [
        'df', 'labels', 'embeddings', 'indices', 'X_train', 'X_test',
        'y_train', 'y_test', 'train_indices', 'test_indices',
        'baseline_model', 'final_model', 'study',
    ]
    for var_name in vars_to_delete:
        if var_name in locals() or var_name in globals():
            if var_name in globals(): del globals()[var_name]

    gc.collect()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")

Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 97108건, 테스트용: 24278건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-11 21:03:15,449] A new study created in memory with name: no-name-747389dd-ac48-4b46-b797-bfd2b661380f


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...
(최대 50번 시도, 10번 개선 없으면 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-11 21:03:39,472] Trial 0 finished with value: 0.18111642941895523 and parameters: {'n_estimators': 500, 'learning_rate': 0.058094245451650074, 'num_leaves': 285, 'max_depth': 9, 'lambda_l1': 0.001310261415545032, 'lambda_l2': 0.01040080217542489, 'feature_fraction': 0.9463124922493077, 'bagging_fraction': 0.7031642813235403, 'bagging_freq': 6}. Best is trial 0 with value: 0.18111642941895523.
[I 2025-10-11 21:04:05,603] Trial 1 finished with value: 0.1998090152976513 and parameters: {'n_estimators': 700, 'learning_rate': 0.0036518647659001197, 'num_leaves': 27, 'max_depth': 10, 'lambda_l1': 2.283171361851737e-08, 'lambda_l2': 6.151903428525071e-08, 'feature_fraction': 0.9859484792615983, 'bagging_fraction': 0.7756480402219587, 'bagging_freq': 4}. Best is trial 1 with value: 0.1998090152976513.
[I 2025-10-11 21:04:23,647] Trial 2 finished with value: 0.2078844121121917 and parameters: {'n_estimators': 1100, 'learning_rate': 0.054114441477715575, 'num_leaves': 300, 'max_depth'


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 1100
       learning_rate: 0.054114441477715575
          num_leaves: 300
           max_depth: 7
           lambda_l1: 8.681406465477098
           lambda_l2: 2.4854725646545834e-06
    feature_fraction: 0.9461824632394943
    bagging_fraction: 0.7161866797991427
        bagging_freq: 7

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1846   0.7771    0.0701
Optuna Tuned  0.1944   0.7709    0.0788

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/DistilBERT_coursera_with_s2_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 7: 사용된 변수들을 메모리에서 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


# Audible

## T5

In [ ]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 입력 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/audible/audible.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/audible_T5.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/T5_audible_with_s2_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    EARLY_STOPPING_ROUNDS = 10 # 🌟 Optuna 조기 종료 횟수

# === 3. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    """Optuna 스터디의 조기 종료를 위한 콜백 클래스"""
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()

# === 4. Optuna Objective 함수 ===
def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    params = {
        'objective': 'binary', 'metric': 'logloss', 'verbosity': -1,
        'boosting_type': 'gbdt', 'random_state': Config.RANDOM_STATE,
        'device': 'gpu',  # 🌟 Optuna Trial 내부에서도 GPU 사용
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              eval_metric='logloss',
              callbacks=[lgb.early_stopping(100, verbose=False)])

    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)
    return score

# === 5. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # ... (Step 1 ~ 2: 데이터 로드, 분할, 베이스라인 측정) ...
    print("Step 1: 데이터 로드 및 분할 중...")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
        assert len(df) == len(embeddings)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
    )
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    baseline_model = lgb.LGBMClassifier(device='gpu', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 (조기 종료 포함) ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.EARLY_STOPPING_ROUNDS}번 개선 없으면 조기 종료)")
    print("="*50)

    early_stopping_callback = EarlyStoppingCallback(early_stopping_rounds=config.EARLY_STOPPING_ROUNDS)
    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda study, trial: pbar.update(1), early_stopping_callback])
    except optuna.exceptions.OptunaError:
        # 조기 종료 시 발생하는 예외를 정상 처리
        pass
    pbar.close()

    # ... (Step 4 ~ 6: 결과 출력, 저장, 메모리 정리) ...
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    final_model = lgb.LGBMClassifier(device='gpu', objective='binary', verbosity=-1, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    df['s2_pred_proba'] = np.nan
    df['s2_pred_class'] = np.nan

    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[train_indices, 's2_pred_class'] = train_pred_class

    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned
    df.loc[test_indices, 's2_pred_class'] = y_pred_class_tuned

    # 🌟 Config에 설정된 경로로 최종 파일 저장
    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    print("\n" + "="*60)
    print("🧹 Step 7: 사용된 변수들을 메모리에서 정리")
    print("="*60)

    vars_to_delete = [
        'df', 'labels', 'embeddings', 'indices', 'X_train', 'X_test',
        'y_train', 'y_test', 'train_indices', 'test_indices',
        'baseline_model', 'final_model', 'study',
    ]
    for var_name in vars_to_delete:
        if var_name in locals() or var_name in globals():
            if var_name in globals(): del globals()[var_name]

    gc.collect()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")

Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 74391건, 테스트용: 18598건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-11 21:09:24,924] A new study created in memory with name: no-name-8d095473-fe9b-4213-8d93-05404b4f623b


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...
(최대 50번 시도, 10번 개선 없으면 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-11 21:10:56,660] Trial 0 finished with value: 0.1615461098561281 and parameters: {'n_estimators': 1900, 'learning_rate': 0.0018579444209650842, 'num_leaves': 46, 'max_depth': 11, 'lambda_l1': 2.3721396579494408e-05, 'lambda_l2': 5.656955078244137, 'feature_fraction': 0.7956508263749972, 'bagging_fraction': 0.7010694270666304, 'bagging_freq': 6}. Best is trial 0 with value: 0.1615461098561281.
[I 2025-10-11 21:11:04,457] Trial 1 finished with value: 0.1528516809693229 and parameters: {'n_estimators': 600, 'learning_rate': 0.03332928760418356, 'num_leaves': 205, 'max_depth': 3, 'lambda_l1': 9.047077665292781e-06, 'lambda_l2': 0.12909842284352258, 'feature_fraction': 0.9730000601078201, 'bagging_fraction': 0.7628127238052675, 'bagging_freq': 1}. Best is trial 0 with value: 0.1615461098561281.
[I 2025-10-11 21:11:12,531] Trial 2 finished with value: 0.15353316899212915 and parameters: {'n_estimators': 1500, 'learning_rate': 0.057215948070315055, 'num_leaves': 131, 'max_depth': 5


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 1000
       learning_rate: 0.004451578538008239
          num_leaves: 74
           max_depth: 9
           lambda_l1: 0.48131928307825345
           lambda_l2: 0.025837136669473012
    feature_fraction: 0.863991217155609
    bagging_fraction: 0.9372161621462469
        bagging_freq: 7

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1575   0.6698    0.0087
Optuna Tuned  0.1716   0.6845    0.0000

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/T5_audible_with_s2_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 7: 사용된 변수들을 메모리에서 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## BERT

In [ ]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 입력 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/audible/audible.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/audible_BERT.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/BERT_audible_with_s2_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    EARLY_STOPPING_ROUNDS = 10 # 🌟 Optuna 조기 종료 횟수

# === 3. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    """Optuna 스터디의 조기 종료를 위한 콜백 클래스"""
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()

# === 4. Optuna Objective 함수 ===
def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    params = {
        'objective': 'binary', 'metric': 'logloss', 'verbosity': -1,
        'boosting_type': 'gbdt', 'random_state': Config.RANDOM_STATE,
        'device': 'gpu',  # 🌟 Optuna Trial 내부에서도 GPU 사용
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              eval_metric='logloss',
              callbacks=[lgb.early_stopping(100, verbose=False)])

    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)
    return score

# === 5. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # ... (Step 1 ~ 2: 데이터 로드, 분할, 베이스라인 측정) ...
    print("Step 1: 데이터 로드 및 분할 중...")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
        assert len(df) == len(embeddings)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
    )
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    baseline_model = lgb.LGBMClassifier(device='gpu', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 (조기 종료 포함) ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.EARLY_STOPPING_ROUNDS}번 개선 없으면 조기 종료)")
    print("="*50)

    early_stopping_callback = EarlyStoppingCallback(early_stopping_rounds=config.EARLY_STOPPING_ROUNDS)
    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda study, trial: pbar.update(1), early_stopping_callback])
    except optuna.exceptions.OptunaError:
        # 조기 종료 시 발생하는 예외를 정상 처리
        pass
    pbar.close()

    # ... (Step 4 ~ 6: 결과 출력, 저장, 메모리 정리) ...
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    final_model = lgb.LGBMClassifier(device='gpu', objective='binary', verbosity=-1, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    df['s2_pred_proba'] = np.nan
    df['s2_pred_class'] = np.nan

    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[train_indices, 's2_pred_class'] = train_pred_class

    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned
    df.loc[test_indices, 's2_pred_class'] = y_pred_class_tuned

    # 🌟 Config에 설정된 경로로 최종 파일 저장
    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    print("\n" + "="*60)
    print("🧹 Step 7: 사용된 변수들을 메모리에서 정리")
    print("="*60)

    vars_to_delete = [
        'df', 'labels', 'embeddings', 'indices', 'X_train', 'X_test',
        'y_train', 'y_test', 'train_indices', 'test_indices',
        'baseline_model', 'final_model', 'study',
    ]
    for var_name in vars_to_delete:
        if var_name in locals() or var_name in globals():
            if var_name in globals(): del globals()[var_name]

    gc.collect()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")

Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 74391건, 테스트용: 18598건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-11 21:19:53,414] A new study created in memory with name: no-name-dc33f4cb-91d2-49e2-8852-9775545cf8dd


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...
(최대 50번 시도, 10번 개선 없으면 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-11 21:20:05,602] Trial 0 finished with value: 0.141489347632069 and parameters: {'n_estimators': 2000, 'learning_rate': 0.054895265457821786, 'num_leaves': 107, 'max_depth': 7, 'lambda_l1': 0.004888051041968124, 'lambda_l2': 1.0519181140301666e-07, 'feature_fraction': 0.9077283755222015, 'bagging_fraction': 0.8228296331058187, 'bagging_freq': 2}. Best is trial 0 with value: 0.141489347632069.
[I 2025-10-11 21:20:18,506] Trial 1 finished with value: 0.1298104509563271 and parameters: {'n_estimators': 300, 'learning_rate': 0.09550865772153064, 'num_leaves': 93, 'max_depth': 8, 'lambda_l1': 0.0034754729926163947, 'lambda_l2': 3.207493039666788e-08, 'feature_fraction': 0.9110144518733756, 'bagging_fraction': 0.9266395446871918, 'bagging_freq': 5}. Best is trial 0 with value: 0.141489347632069.
[I 2025-10-11 21:21:20,618] Trial 2 finished with value: 0.1423933619722336 and parameters: {'n_estimators': 1200, 'learning_rate': 0.008036989670498467, 'num_leaves': 221, 'max_depth': 12


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 300
       learning_rate: 0.03592280632071865
          num_leaves: 285
           max_depth: 4
           lambda_l1: 6.868642672596015
           lambda_l2: 1.1929083069353505e-05
    feature_fraction: 0.8209357514611245
    bagging_fraction: 0.9601977186143064
        bagging_freq: 6

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline       0.135   0.6420    0.0086
Optuna Tuned   0.152   0.6594    0.0000

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/BERT_audible_with_s2_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 7: 사용된 변수들을 메모리에서 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## SentenceBERT

In [ ]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 입력 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/audible/audible.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/audible_SentenceBERT.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/SentenceBERT_audible_with_s2_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    EARLY_STOPPING_ROUNDS = 10 # 🌟 Optuna 조기 종료 횟수

# === 3. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    """Optuna 스터디의 조기 종료를 위한 콜백 클래스"""
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()

# === 4. Optuna Objective 함수 ===
def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    params = {
        'objective': 'binary', 'metric': 'logloss', 'verbosity': -1,
        'boosting_type': 'gbdt', 'random_state': Config.RANDOM_STATE,
        'device': 'gpu',  # 🌟 Optuna Trial 내부에서도 GPU 사용
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              eval_metric='logloss',
              callbacks=[lgb.early_stopping(100, verbose=False)])

    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)
    return score

# === 5. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # ... (Step 1 ~ 2: 데이터 로드, 분할, 베이스라인 측정) ...
    print("Step 1: 데이터 로드 및 분할 중...")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
        assert len(df) == len(embeddings)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
    )
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    baseline_model = lgb.LGBMClassifier(device='gpu', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 (조기 종료 포함) ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.EARLY_STOPPING_ROUNDS}번 개선 없으면 조기 종료)")
    print("="*50)

    early_stopping_callback = EarlyStoppingCallback(early_stopping_rounds=config.EARLY_STOPPING_ROUNDS)
    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda study, trial: pbar.update(1), early_stopping_callback])
    except optuna.exceptions.OptunaError:
        # 조기 종료 시 발생하는 예외를 정상 처리
        pass
    pbar.close()

    # ... (Step 4 ~ 6: 결과 출력, 저장, 메모리 정리) ...
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    final_model = lgb.LGBMClassifier(device='gpu', objective='binary', verbosity=-1, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    df['s2_pred_proba'] = np.nan
    df['s2_pred_class'] = np.nan

    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[train_indices, 's2_pred_class'] = train_pred_class

    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned
    df.loc[test_indices, 's2_pred_class'] = y_pred_class_tuned

    # 🌟 Config에 설정된 경로로 최종 파일 저장
    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    print("\n" + "="*60)
    print("🧹 Step 7: 사용된 변수들을 메모리에서 정리")
    print("="*60)

    vars_to_delete = [
        'df', 'labels', 'embeddings', 'indices', 'X_train', 'X_test',
        'y_train', 'y_test', 'train_indices', 'test_indices',
        'baseline_model', 'final_model', 'study',
    ]
    for var_name in vars_to_delete:
        if var_name in locals() or var_name in globals():
            if var_name in globals(): del globals()[var_name]

    gc.collect()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")

Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 74391건, 테스트용: 18598건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-11 21:29:04,206] A new study created in memory with name: no-name-59fb69a6-297b-43d4-8896-1f6758a646b6


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...
(최대 50번 시도, 10번 개선 없으면 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-11 21:30:43,470] Trial 0 finished with value: 0.14553675282946257 and parameters: {'n_estimators': 1900, 'learning_rate': 0.0013714659998085848, 'num_leaves': 125, 'max_depth': 9, 'lambda_l1': 0.8996622270722991, 'lambda_l2': 4.640650486289767e-05, 'feature_fraction': 0.7888079403803553, 'bagging_fraction': 0.69469079489932, 'bagging_freq': 3}. Best is trial 0 with value: 0.14553675282946257.
[I 2025-10-11 21:32:39,406] Trial 1 finished with value: 0.14227756526976792 and parameters: {'n_estimators': 1800, 'learning_rate': 0.0017375230342082273, 'num_leaves': 169, 'max_depth': 11, 'lambda_l1': 2.1522814846585543e-05, 'lambda_l2': 1.0076576546200888e-07, 'feature_fraction': 0.7709798732527592, 'bagging_fraction': 0.9247425966141084, 'bagging_freq': 7}. Best is trial 0 with value: 0.14553675282946257.
[I 2025-10-11 21:33:04,388] Trial 2 finished with value: 0.1448184325139152 and parameters: {'n_estimators': 1400, 'learning_rate': 0.01305626539643458, 'num_leaves': 284, 'max_d


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 1900
       learning_rate: 0.0013714659998085848
          num_leaves: 125
           max_depth: 9
           lambda_l1: 0.8996622270722991
           lambda_l2: 4.640650486289767e-05
    feature_fraction: 0.7888079403803553
    bagging_fraction: 0.69469079489932
        bagging_freq: 3

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1353   0.6506    0.0015
Optuna Tuned  0.1600   0.6729    0.0000

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/SentenceBERT_audible_with_s2_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 7: 사용된 변수들을 메모리에서 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## RoBERTa

In [ ]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 입력 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/audible/audible.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/audible_RoBERTa.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/RoBERTa_audible_with_s2_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    EARLY_STOPPING_ROUNDS = 10 # 🌟 Optuna 조기 종료 횟수

# === 3. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    """Optuna 스터디의 조기 종료를 위한 콜백 클래스"""
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()

# === 4. Optuna Objective 함수 ===
def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    params = {
        'objective': 'binary', 'metric': 'logloss', 'verbosity': -1,
        'boosting_type': 'gbdt', 'random_state': Config.RANDOM_STATE,
        'device': 'gpu',  # 🌟 Optuna Trial 내부에서도 GPU 사용
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              eval_metric='logloss',
              callbacks=[lgb.early_stopping(100, verbose=False)])

    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)
    return score

# === 5. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # ... (Step 1 ~ 2: 데이터 로드, 분할, 베이스라인 측정) ...
    print("Step 1: 데이터 로드 및 분할 중...")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
        assert len(df) == len(embeddings)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
    )
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    baseline_model = lgb.LGBMClassifier(device='gpu', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 (조기 종료 포함) ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.EARLY_STOPPING_ROUNDS}번 개선 없으면 조기 종료)")
    print("="*50)

    early_stopping_callback = EarlyStoppingCallback(early_stopping_rounds=config.EARLY_STOPPING_ROUNDS)
    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda study, trial: pbar.update(1), early_stopping_callback])
    except optuna.exceptions.OptunaError:
        # 조기 종료 시 발생하는 예외를 정상 처리
        pass
    pbar.close()

    # ... (Step 4 ~ 6: 결과 출력, 저장, 메모리 정리) ...
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    final_model = lgb.LGBMClassifier(device='gpu', objective='binary', verbosity=-1, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    df['s2_pred_proba'] = np.nan
    df['s2_pred_class'] = np.nan

    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[train_indices, 's2_pred_class'] = train_pred_class

    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned
    df.loc[test_indices, 's2_pred_class'] = y_pred_class_tuned

    # 🌟 Config에 설정된 경로로 최종 파일 저장
    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    print("\n" + "="*60)
    print("🧹 Step 7: 사용된 변수들을 메모리에서 정리")
    print("="*60)

    vars_to_delete = [
        'df', 'labels', 'embeddings', 'indices', 'X_train', 'X_test',
        'y_train', 'y_test', 'train_indices', 'test_indices',
        'baseline_model', 'final_model', 'study',
    ]
    for var_name in vars_to_delete:
        if var_name in locals() or var_name in globals():
            if var_name in globals(): del globals()[var_name]

    gc.collect()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")

Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 74391건, 테스트용: 18598건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-11 21:36:33,685] A new study created in memory with name: no-name-e2d1dcd3-2271-4e79-b1cf-0eaabef7efd4


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...
(최대 50번 시도, 10번 개선 없으면 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-11 21:36:40,640] Trial 0 finished with value: 0.1415243894266757 and parameters: {'n_estimators': 400, 'learning_rate': 0.02135444366250037, 'num_leaves': 20, 'max_depth': 3, 'lambda_l1': 1.3616801391692733e-06, 'lambda_l2': 0.09194350421130441, 'feature_fraction': 0.9403636574647329, 'bagging_fraction': 0.6980357963491699, 'bagging_freq': 5}. Best is trial 0 with value: 0.1415243894266757.
[I 2025-10-11 21:36:48,741] Trial 1 finished with value: 0.1351579884352608 and parameters: {'n_estimators': 900, 'learning_rate': 0.0721751286437755, 'num_leaves': 20, 'max_depth': 11, 'lambda_l1': 0.032751189551571386, 'lambda_l2': 4.2292213123740994e-05, 'feature_fraction': 0.9297496418709909, 'bagging_fraction': 0.9686517532180405, 'bagging_freq': 1}. Best is trial 0 with value: 0.1415243894266757.
[I 2025-10-11 21:37:11,458] Trial 2 finished with value: 0.14843502787557472 and parameters: {'n_estimators': 1100, 'learning_rate': 0.011816379893566027, 'num_leaves': 146, 'max_depth': 6,


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 1600
       learning_rate: 0.0073225519424753175
          num_leaves: 168
           max_depth: 5
           lambda_l1: 8.499825408574972
           lambda_l2: 5.968419966588605e-08
    feature_fraction: 0.6675978590053072
    bagging_fraction: 0.8124284314376489
        bagging_freq: 2

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1395   0.6480    0.0101
Optuna Tuned  0.1546   0.6666    0.0015

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/RoBERTa_audible_with_s2_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 7: 사용된 변수들을 메모리에서 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## DistilBERT

In [ ]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 입력 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/audible/audible.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/audible_DistilBERT.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/DistilBERT_audible_with_s2_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    EARLY_STOPPING_ROUNDS = 10 # 🌟 Optuna 조기 종료 횟수

# === 3. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    """Optuna 스터디의 조기 종료를 위한 콜백 클래스"""
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()

# === 4. Optuna Objective 함수 ===
def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    params = {
        'objective': 'binary', 'metric': 'logloss', 'verbosity': -1,
        'boosting_type': 'gbdt', 'random_state': Config.RANDOM_STATE,
        'device': 'gpu',  # 🌟 Optuna Trial 내부에서도 GPU 사용
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              eval_metric='logloss',
              callbacks=[lgb.early_stopping(100, verbose=False)])

    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)
    return score

# === 5. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # ... (Step 1 ~ 2: 데이터 로드, 분할, 베이스라인 측정) ...
    print("Step 1: 데이터 로드 및 분할 중...")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
        assert len(df) == len(embeddings)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
    )
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    baseline_model = lgb.LGBMClassifier(device='gpu', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 (조기 종료 포함) ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.EARLY_STOPPING_ROUNDS}번 개선 없으면 조기 종료)")
    print("="*50)

    early_stopping_callback = EarlyStoppingCallback(early_stopping_rounds=config.EARLY_STOPPING_ROUNDS)
    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda study, trial: pbar.update(1), early_stopping_callback])
    except optuna.exceptions.OptunaError:
        # 조기 종료 시 발생하는 예외를 정상 처리
        pass
    pbar.close()

    # ... (Step 4 ~ 6: 결과 출력, 저장, 메모리 정리) ...
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    final_model = lgb.LGBMClassifier(device='gpu', objective='binary', verbosity=-1, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    df['s2_pred_proba'] = np.nan
    df['s2_pred_class'] = np.nan

    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[train_indices, 's2_pred_class'] = train_pred_class

    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned
    df.loc[test_indices, 's2_pred_class'] = y_pred_class_tuned

    # 🌟 Config에 설정된 경로로 최종 파일 저장
    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    print("\n" + "="*60)
    print("🧹 Step 7: 사용된 변수들을 메모리에서 정리")
    print("="*60)

    vars_to_delete = [
        'df', 'labels', 'embeddings', 'indices', 'X_train', 'X_test',
        'y_train', 'y_test', 'train_indices', 'test_indices',
        'baseline_model', 'final_model', 'study',
    ]
    for var_name in vars_to_delete:
        if var_name in locals() or var_name in globals():
            if var_name in globals(): del globals()[var_name]

    gc.collect()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")

Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 74391건, 테스트용: 18598건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-11 22:01:01,129] A new study created in memory with name: no-name-a15d3f70-6f56-4ad3-be3a-41a13427900b


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...
(최대 50번 시도, 10번 개선 없으면 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-11 22:01:12,329] Trial 0 finished with value: 0.14688707822727604 and parameters: {'n_estimators': 2000, 'learning_rate': 0.021727139532555003, 'num_leaves': 88, 'max_depth': 5, 'lambda_l1': 6.798930373100897e-05, 'lambda_l2': 7.487895916827592, 'feature_fraction': 0.8393529020860595, 'bagging_fraction': 0.7975251112943258, 'bagging_freq': 5}. Best is trial 0 with value: 0.14688707822727604.
[I 2025-10-11 22:02:08,532] Trial 1 finished with value: 0.14198449611236572 and parameters: {'n_estimators': 1700, 'learning_rate': 0.0014311790934840038, 'num_leaves': 31, 'max_depth': 11, 'lambda_l1': 0.01141525217302734, 'lambda_l2': 0.00018458535407303583, 'feature_fraction': 0.8329413573837661, 'bagging_fraction': 0.9881747852137114, 'bagging_freq': 4}. Best is trial 0 with value: 0.14688707822727604.
[I 2025-10-11 22:04:27,628] Trial 2 finished with value: 0.14663525715032577 and parameters: {'n_estimators': 1900, 'learning_rate': 0.0020766899003741334, 'num_leaves': 199, 'max_dep


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 2000
       learning_rate: 0.021727139532555003
          num_leaves: 88
           max_depth: 5
           lambda_l1: 6.798930373100897e-05
           lambda_l2: 7.487895916827592
    feature_fraction: 0.8393529020860595
    bagging_fraction: 0.7975251112943258
        bagging_freq: 5

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1443   0.6546    0.0044
Optuna Tuned  0.1586   0.6582    0.0044

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/DistilBERT_audible_with_s2_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 7: 사용된 변수들을 메모리에서 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


# Hotel

## T5

In [5]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 입력 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/hotel/hotel.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/hotel_T5.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/T5_hotel_with_s2_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    EARLY_STOPPING_ROUNDS = 10 # 🌟 Optuna 조기 종료 횟수

# === 3. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    """Optuna 스터디의 조기 종료를 위한 콜백 클래스"""
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()

# === 4. Optuna Objective 함수 ===
def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    params = {
        'objective': 'binary', 'metric': 'logloss', 'verbosity': -1,
        'boosting_type': 'gbdt', 'random_state': Config.RANDOM_STATE,
        'device': 'gpu',  # 🌟 Optuna Trial 내부에서도 GPU 사용
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              eval_metric='logloss',
              callbacks=[lgb.early_stopping(100, verbose=False)])

    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)
    return score

# === 5. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # ... (Step 1 ~ 2: 데이터 로드, 분할, 베이스라인 측정) ...
    print("Step 1: 데이터 로드 및 분할 중...")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
        assert len(df) == len(embeddings)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
    )
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    baseline_model = lgb.LGBMClassifier(device='gpu', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 (조기 종료 포함) ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.EARLY_STOPPING_ROUNDS}번 개선 없으면 조기 종료)")
    print("="*50)

    early_stopping_callback = EarlyStoppingCallback(early_stopping_rounds=config.EARLY_STOPPING_ROUNDS)
    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda study, trial: pbar.update(1), early_stopping_callback])
    except optuna.exceptions.OptunaError:
        # 조기 종료 시 발생하는 예외를 정상 처리
        pass
    pbar.close()

    # ... (Step 4 ~ 6: 결과 출력, 저장, 메모리 정리) ...
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    final_model = lgb.LGBMClassifier(device='gpu', objective='binary', verbosity=-1, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    df['s2_pred_proba'] = np.nan
    df['s2_pred_class'] = np.nan

    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[train_indices, 's2_pred_class'] = train_pred_class

    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned
    df.loc[test_indices, 's2_pred_class'] = y_pred_class_tuned

    # 🌟 Config에 설정된 경로로 최종 파일 저장
    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    print("\n" + "="*60)
    print("🧹 Step 7: 사용된 변수들을 메모리에서 정리")
    print("="*60)

    vars_to_delete = [
        'df', 'labels', 'embeddings', 'indices', 'X_train', 'X_test',
        'y_train', 'y_test', 'train_indices', 'test_indices',
        'baseline_model', 'final_model', 'study',
    ]
    for var_name in vars_to_delete:
        if var_name in locals() or var_name in globals():
            if var_name in globals(): del globals()[var_name]

    gc.collect()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")

Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71604건, 테스트용: 17901건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)
[LightGBM] [Info] Number of positive: 6586, number of negative: 65018
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 195840
[LightGBM] [Info] Number of data points in the train set: 71604, number of used features: 768
[LightGBM] [Info] Using GPU Device: NVIDIA A100-SXM4-80GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 768 dense feature groups (52.44 MB) transferred to GPU in 0.019890 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.091978 -> initscore=-2.289718
[LightGBM] [Info] Start training from score -2.289718


[I 2025-10-12 06:26:35,031] A new study created in memory with name: no-name-7340e22d-8b6d-4ec2-9e20-2a50f926cf95


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...
(최대 50번 시도, 10번 개선 없으면 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 06:26:45,198] Trial 0 finished with value: 0.15213446515927667 and parameters: {'n_estimators': 1200, 'learning_rate': 0.020417841187667107, 'num_leaves': 39, 'max_depth': 5, 'lambda_l1': 0.4792098310469275, 'lambda_l2': 1.078154277561877, 'feature_fraction': 0.7916919694550321, 'bagging_fraction': 0.7800569871647935, 'bagging_freq': 6}. Best is trial 0 with value: 0.15213446515927667.
[I 2025-10-12 06:27:05,358] Trial 1 finished with value: 0.14432457209665023 and parameters: {'n_estimators': 500, 'learning_rate': 0.028070308628647355, 'num_leaves': 155, 'max_depth': 11, 'lambda_l1': 0.00032401533379208006, 'lambda_l2': 1.9981262241385757e-06, 'feature_fraction': 0.7258776302053424, 'bagging_fraction': 0.6821596666513379, 'bagging_freq': 4}. Best is trial 0 with value: 0.15213446515927667.
[I 2025-10-12 06:27:18,210] Trial 2 finished with value: 0.120059808064244 and parameters: {'n_estimators': 1900, 'learning_rate': 0.22668538456619708, 'num_leaves': 236, 'max_depth': 


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 1800
       learning_rate: 0.002481033466505058
          num_leaves: 273
           max_depth: 8
           lambda_l1: 1.3420600252507857
           lambda_l2: 0.00845504228759606
    feature_fraction: 0.6288263588306325
    bagging_fraction: 0.9424137919143921
        bagging_freq: 5

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1390   0.6099       0.0
Optuna Tuned  0.1648   0.6356       0.0

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/T5_hotel_with_s2_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 7: 사용된 변수들을 메모리에서 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## BERT

In [6]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 입력 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/hotel/hotel.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/hotel_BERT.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/BERT_hotel_with_s2_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    EARLY_STOPPING_ROUNDS = 10 # 🌟 Optuna 조기 종료 횟수

# === 3. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    """Optuna 스터디의 조기 종료를 위한 콜백 클래스"""
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()

# === 4. Optuna Objective 함수 ===
def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    params = {
        'objective': 'binary', 'metric': 'logloss', 'verbosity': -1,
        'boosting_type': 'gbdt', 'random_state': Config.RANDOM_STATE,
        'device': 'gpu',  # 🌟 Optuna Trial 내부에서도 GPU 사용
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              eval_metric='logloss',
              callbacks=[lgb.early_stopping(100, verbose=False)])

    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)
    return score

# === 5. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # ... (Step 1 ~ 2: 데이터 로드, 분할, 베이스라인 측정) ...
    print("Step 1: 데이터 로드 및 분할 중...")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
        assert len(df) == len(embeddings)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
    )
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    baseline_model = lgb.LGBMClassifier(device='gpu', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 (조기 종료 포함) ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.EARLY_STOPPING_ROUNDS}번 개선 없으면 조기 종료)")
    print("="*50)

    early_stopping_callback = EarlyStoppingCallback(early_stopping_rounds=config.EARLY_STOPPING_ROUNDS)
    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda study, trial: pbar.update(1), early_stopping_callback])
    except optuna.exceptions.OptunaError:
        # 조기 종료 시 발생하는 예외를 정상 처리
        pass
    pbar.close()

    # ... (Step 4 ~ 6: 결과 출력, 저장, 메모리 정리) ...
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    final_model = lgb.LGBMClassifier(device='gpu', objective='binary', verbosity=-1, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    df['s2_pred_proba'] = np.nan
    df['s2_pred_class'] = np.nan

    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[train_indices, 's2_pred_class'] = train_pred_class

    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned
    df.loc[test_indices, 's2_pred_class'] = y_pred_class_tuned

    # 🌟 Config에 설정된 경로로 최종 파일 저장
    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    print("\n" + "="*60)
    print("🧹 Step 7: 사용된 변수들을 메모리에서 정리")
    print("="*60)

    vars_to_delete = [
        'df', 'labels', 'embeddings', 'indices', 'X_train', 'X_test',
        'y_train', 'y_test', 'train_indices', 'test_indices',
        'baseline_model', 'final_model', 'study',
    ]
    for var_name in vars_to_delete:
        if var_name in locals() or var_name in globals():
            if var_name in globals(): del globals()[var_name]

    gc.collect()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")

Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71604건, 테스트용: 17901건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 06:37:07,949] A new study created in memory with name: no-name-ff8f9b83-1323-4efc-abf9-6482afa7b345


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...
(최대 50번 시도, 10번 개선 없으면 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 06:37:16,230] Trial 0 finished with value: 0.1261756742383485 and parameters: {'n_estimators': 2000, 'learning_rate': 0.28616219469207493, 'num_leaves': 62, 'max_depth': 9, 'lambda_l1': 8.863677188401592e-08, 'lambda_l2': 1.2632018974413965e-08, 'feature_fraction': 0.7124478477033773, 'bagging_fraction': 0.6716822014072272, 'bagging_freq': 7}. Best is trial 0 with value: 0.1261756742383485.
[I 2025-10-12 06:37:21,972] Trial 1 finished with value: 0.15003664805482397 and parameters: {'n_estimators': 800, 'learning_rate': 0.16895044706987095, 'num_leaves': 171, 'max_depth': 4, 'lambda_l1': 0.0026519164972713754, 'lambda_l2': 0.5692871528067157, 'feature_fraction': 0.7215793033500075, 'bagging_fraction': 0.9521355676770022, 'bagging_freq': 6}. Best is trial 1 with value: 0.15003664805482397.
[I 2025-10-12 06:38:04,758] Trial 2 finished with value: 0.1511834151167769 and parameters: {'n_estimators': 600, 'learning_rate': 0.0010879785020850672, 'num_leaves': 72, 'max_depth': 1


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 1500
       learning_rate: 0.002830668835820194
          num_leaves: 26
           max_depth: 5
           lambda_l1: 9.231545233803382e-05
           lambda_l2: 0.002879832751125398
    feature_fraction: 0.6501494595468753
    bagging_fraction: 0.6050573502906857
        bagging_freq: 4

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1463   0.6203    0.0024
Optuna Tuned  0.1600   0.6294    0.0000

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/BERT_hotel_with_s2_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 7: 사용된 변수들을 메모리에서 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## SentenceBERT

In [7]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 입력 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/hotel/hotel.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/hotel_SentenceBERT.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/SentenceBERT_hotel_with_s2_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    EARLY_STOPPING_ROUNDS = 10 # 🌟 Optuna 조기 종료 횟수

# === 3. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    """Optuna 스터디의 조기 종료를 위한 콜백 클래스"""
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()

# === 4. Optuna Objective 함수 ===
def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    params = {
        'objective': 'binary', 'metric': 'logloss', 'verbosity': -1,
        'boosting_type': 'gbdt', 'random_state': Config.RANDOM_STATE,
        'device': 'gpu',  # 🌟 Optuna Trial 내부에서도 GPU 사용
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              eval_metric='logloss',
              callbacks=[lgb.early_stopping(100, verbose=False)])

    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)
    return score

# === 5. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # ... (Step 1 ~ 2: 데이터 로드, 분할, 베이스라인 측정) ...
    print("Step 1: 데이터 로드 및 분할 중...")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
        assert len(df) == len(embeddings)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
    )
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    baseline_model = lgb.LGBMClassifier(device='gpu', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 (조기 종료 포함) ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.EARLY_STOPPING_ROUNDS}번 개선 없으면 조기 종료)")
    print("="*50)

    early_stopping_callback = EarlyStoppingCallback(early_stopping_rounds=config.EARLY_STOPPING_ROUNDS)
    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda study, trial: pbar.update(1), early_stopping_callback])
    except optuna.exceptions.OptunaError:
        # 조기 종료 시 발생하는 예외를 정상 처리
        pass
    pbar.close()

    # ... (Step 4 ~ 6: 결과 출력, 저장, 메모리 정리) ...
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    final_model = lgb.LGBMClassifier(device='gpu', objective='binary', verbosity=-1, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    df['s2_pred_proba'] = np.nan
    df['s2_pred_class'] = np.nan

    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[train_indices, 's2_pred_class'] = train_pred_class

    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned
    df.loc[test_indices, 's2_pred_class'] = y_pred_class_tuned

    # 🌟 Config에 설정된 경로로 최종 파일 저장
    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    print("\n" + "="*60)
    print("🧹 Step 7: 사용된 변수들을 메모리에서 정리")
    print("="*60)

    vars_to_delete = [
        'df', 'labels', 'embeddings', 'indices', 'X_train', 'X_test',
        'y_train', 'y_test', 'train_indices', 'test_indices',
        'baseline_model', 'final_model', 'study',
    ]
    for var_name in vars_to_delete:
        if var_name in locals() or var_name in globals():
            if var_name in globals(): del globals()[var_name]

    gc.collect()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")

Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71604건, 테스트용: 17901건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 06:47:53,156] A new study created in memory with name: no-name-c743a3a7-dc01-456c-9778-e91dd5d31aac


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...
(최대 50번 시도, 10번 개선 없으면 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 06:47:56,546] Trial 0 finished with value: 0.13717886619943928 and parameters: {'n_estimators': 500, 'learning_rate': 0.1152319195237506, 'num_leaves': 101, 'max_depth': 5, 'lambda_l1': 7.719021570410522e-06, 'lambda_l2': 1.2163945012053485e-06, 'feature_fraction': 0.8868738973372874, 'bagging_fraction': 0.6801277396230722, 'bagging_freq': 1}. Best is trial 0 with value: 0.13717886619943928.
[I 2025-10-12 06:48:07,202] Trial 1 finished with value: 0.14710225707341368 and parameters: {'n_estimators': 700, 'learning_rate': 0.004001936614871667, 'num_leaves': 23, 'max_depth': 9, 'lambda_l1': 0.5682348845098928, 'lambda_l2': 1.5117010187302793e-07, 'feature_fraction': 0.7822946318094723, 'bagging_fraction': 0.789935867639714, 'bagging_freq': 1}. Best is trial 1 with value: 0.14710225707341368.
[I 2025-10-12 06:49:28,978] Trial 2 finished with value: 0.14380653328111323 and parameters: {'n_estimators': 800, 'learning_rate': 0.0017815263726160295, 'num_leaves': 297, 'max_depth'


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 1400
       learning_rate: 0.016870397012923487
          num_leaves: 179
           max_depth: 3
           lambda_l1: 0.010418235143693487
           lambda_l2: 2.1948382378677365
    feature_fraction: 0.7305379430644312
    bagging_fraction: 0.6055332408290104
        bagging_freq: 2

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1404   0.6208    0.0024
Optuna Tuned  0.1557   0.6235    0.0012

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/SentenceBERT_hotel_with_s2_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 7: 사용된 변수들을 메모리에서 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## RoBERTa


In [8]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 입력 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/hotel/hotel.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/hotel_RoBERTa.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/RoBERTa_hotel_with_s2_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    EARLY_STOPPING_ROUNDS = 10 # 🌟 Optuna 조기 종료 횟수

# === 3. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    """Optuna 스터디의 조기 종료를 위한 콜백 클래스"""
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()

# === 4. Optuna Objective 함수 ===
def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    params = {
        'objective': 'binary', 'metric': 'logloss', 'verbosity': -1,
        'boosting_type': 'gbdt', 'random_state': Config.RANDOM_STATE,
        'device': 'gpu',  # 🌟 Optuna Trial 내부에서도 GPU 사용
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              eval_metric='logloss',
              callbacks=[lgb.early_stopping(100, verbose=False)])

    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)
    return score

# === 5. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # ... (Step 1 ~ 2: 데이터 로드, 분할, 베이스라인 측정) ...
    print("Step 1: 데이터 로드 및 분할 중...")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
        assert len(df) == len(embeddings)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
    )
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    baseline_model = lgb.LGBMClassifier(device='gpu', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 (조기 종료 포함) ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.EARLY_STOPPING_ROUNDS}번 개선 없으면 조기 종료)")
    print("="*50)

    early_stopping_callback = EarlyStoppingCallback(early_stopping_rounds=config.EARLY_STOPPING_ROUNDS)
    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda study, trial: pbar.update(1), early_stopping_callback])
    except optuna.exceptions.OptunaError:
        # 조기 종료 시 발생하는 예외를 정상 처리
        pass
    pbar.close()

    # ... (Step 4 ~ 6: 결과 출력, 저장, 메모리 정리) ...
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    final_model = lgb.LGBMClassifier(device='gpu', objective='binary', verbosity=-1, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    df['s2_pred_proba'] = np.nan
    df['s2_pred_class'] = np.nan

    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[train_indices, 's2_pred_class'] = train_pred_class

    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned
    df.loc[test_indices, 's2_pred_class'] = y_pred_class_tuned

    # 🌟 Config에 설정된 경로로 최종 파일 저장
    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    print("\n" + "="*60)
    print("🧹 Step 7: 사용된 변수들을 메모리에서 정리")
    print("="*60)

    vars_to_delete = [
        'df', 'labels', 'embeddings', 'indices', 'X_train', 'X_test',
        'y_train', 'y_test', 'train_indices', 'test_indices',
        'baseline_model', 'final_model', 'study',
    ]
    for var_name in vars_to_delete:
        if var_name in locals() or var_name in globals():
            if var_name in globals(): del globals()[var_name]

    gc.collect()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")

Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71604건, 테스트용: 17901건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 06:53:21,353] A new study created in memory with name: no-name-e5b34902-bb81-486d-bcee-47f51ac32e5d


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...
(최대 50번 시도, 10번 개선 없으면 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 06:54:01,492] Trial 0 finished with value: 0.1489438350862692 and parameters: {'n_estimators': 700, 'learning_rate': 0.0013217086542025207, 'num_leaves': 85, 'max_depth': 10, 'lambda_l1': 1.0960833422195425e-05, 'lambda_l2': 8.557152478888471e-06, 'feature_fraction': 0.6066949773719481, 'bagging_fraction': 0.8443557724766394, 'bagging_freq': 7}. Best is trial 0 with value: 0.1489438350862692.
[I 2025-10-12 06:54:16,320] Trial 1 finished with value: 0.14959917409695517 and parameters: {'n_estimators': 1800, 'learning_rate': 0.009085687386416613, 'num_leaves': 69, 'max_depth': 5, 'lambda_l1': 0.001096276607387823, 'lambda_l2': 0.03898924084167889, 'feature_fraction': 0.644606874095696, 'bagging_fraction': 0.8740672766511774, 'bagging_freq': 3}. Best is trial 1 with value: 0.14959917409695517.
[I 2025-10-12 06:54:43,513] Trial 2 finished with value: 0.1495907808976755 and parameters: {'n_estimators': 1400, 'learning_rate': 0.0023740284280010736, 'num_leaves': 267, 'max_depth


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 900
       learning_rate: 0.0039196739107984465
          num_leaves: 167
           max_depth: 8
           lambda_l1: 0.13834670343408717
           lambda_l2: 2.2657808278834583e-05
    feature_fraction: 0.8629208085647815
    bagging_fraction: 0.7965134021553388
        bagging_freq: 2

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1425   0.6096     0.006
Optuna Tuned  0.1545   0.6229     0.000

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/RoBERTa_hotel_with_s2_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 7: 사용된 변수들을 메모리에서 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## DistilBERT

In [9]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 입력 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/hotel/hotel.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/hotel_DistilBERT.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/DistilBERT_hotel_with_s2_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    EARLY_STOPPING_ROUNDS = 10 # 🌟 Optuna 조기 종료 횟수

# === 3. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    """Optuna 스터디의 조기 종료를 위한 콜백 클래스"""
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()

# === 4. Optuna Objective 함수 ===
def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    params = {
        'objective': 'binary', 'metric': 'logloss', 'verbosity': -1,
        'boosting_type': 'gbdt', 'random_state': Config.RANDOM_STATE,
        'device': 'gpu',  # 🌟 Optuna Trial 내부에서도 GPU 사용
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              eval_metric='logloss',
              callbacks=[lgb.early_stopping(100, verbose=False)])

    y_pred_proba = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred_proba)
    return score

# === 5. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # ... (Step 1 ~ 2: 데이터 로드, 분할, 베이스라인 측정) ...
    print("Step 1: 데이터 로드 및 분할 중...")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
        assert len(df) == len(embeddings)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
    )
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    baseline_model = lgb.LGBMClassifier(device='gpu', random_state=config.RANDOM_STATE)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 (조기 종료 포함) ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.EARLY_STOPPING_ROUNDS}번 개선 없으면 조기 종료)")
    print("="*50)

    early_stopping_callback = EarlyStoppingCallback(early_stopping_rounds=config.EARLY_STOPPING_ROUNDS)
    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda study, trial: pbar.update(1), early_stopping_callback])
    except optuna.exceptions.OptunaError:
        # 조기 종료 시 발생하는 예외를 정상 처리
        pass
    pbar.close()

    # ... (Step 4 ~ 6: 결과 출력, 저장, 메모리 정리) ...
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    final_model = lgb.LGBMClassifier(device='gpu', objective='binary', verbosity=-1, random_state=config.RANDOM_STATE, **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    df['s2_pred_proba'] = np.nan
    df['s2_pred_class'] = np.nan

    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[train_indices, 's2_pred_class'] = train_pred_class

    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned
    df.loc[test_indices, 's2_pred_class'] = y_pred_class_tuned

    # 🌟 Config에 설정된 경로로 최종 파일 저장
    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    print("\n" + "="*60)
    print("🧹 Step 7: 사용된 변수들을 메모리에서 정리")
    print("="*60)

    vars_to_delete = [
        'df', 'labels', 'embeddings', 'indices', 'X_train', 'X_test',
        'y_train', 'y_test', 'train_indices', 'test_indices',
        'baseline_model', 'final_model', 'study',
    ]
    for var_name in vars_to_delete:
        if var_name in locals() or var_name in globals():
            if var_name in globals(): del globals()[var_name]

    gc.collect()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")

Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71604건, 테스트용: 17901건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 07:18:10,135] A new study created in memory with name: no-name-48295338-fd4c-4ab1-af28-fa865339f7a3


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...
(최대 50번 시도, 10번 개선 없으면 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 07:20:17,917] Trial 0 finished with value: 0.1521748529026728 and parameters: {'n_estimators': 1300, 'learning_rate': 0.0024485150348445297, 'num_leaves': 144, 'max_depth': 12, 'lambda_l1': 0.002435764204214115, 'lambda_l2': 3.6763147316705385, 'feature_fraction': 0.6970161692651633, 'bagging_fraction': 0.7533485759403914, 'bagging_freq': 7}. Best is trial 0 with value: 0.1521748529026728.
[I 2025-10-12 07:20:46,625] Trial 1 finished with value: 0.14569827235677776 and parameters: {'n_estimators': 1100, 'learning_rate': 0.03531844522788899, 'num_leaves': 189, 'max_depth': 12, 'lambda_l1': 0.08104591707730122, 'lambda_l2': 1.6633126088066197, 'feature_fraction': 0.9178589218049207, 'bagging_fraction': 0.7487415428305433, 'bagging_freq': 3}. Best is trial 0 with value: 0.1521748529026728.
[I 2025-10-12 07:20:53,465] Trial 2 finished with value: 0.15004234659446855 and parameters: {'n_estimators': 100, 'learning_rate': 0.07212590610774523, 'num_leaves': 100, 'max_depth': 6, 


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
        n_estimators: 600
       learning_rate: 0.003479008713147986
          num_leaves: 146
           max_depth: 12
           lambda_l1: 0.06922540741247236
           lambda_l2: 0.0009527167291907011
    feature_fraction: 0.7614063818521524
    bagging_fraction: 0.726514458382889
        bagging_freq: 7

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1479   0.6201    0.0036
Optuna Tuned  0.1585   0.6254    0.0000

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/DistilBERT_hotel_with_s2_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 7: 사용된 변수들을 메모리에서 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!
